In [9]:
import numpy as np
import cv2
import glob
import json
import dlib

In [2]:
participants = glob.glob('dataset/*')
print(participants[0])

dataset/0656f5fe35a54d1589e526a702f578b0


In [3]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

In [5]:
# Number of training examples to use(0-2758)
DATASET_SIZE = len(keys_all)
DEBUG = False

In [10]:
detector = dlib.get_frontal_face_detector()

In [12]:
# Number of images with shape 1280x720
N = 50
for i in range(N):
    key = keys_all[i]
    im = cv2.imread('dataset/' + key)
    if(im.shape == (720, 1280, 3)):
        rects = detector(im, 0)

        for rect in rects:
            # convert dlib's rectangle to a OpenCV-style bounding box
            # [i.e., (x, y, w, h)], then draw the face bounding box
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow('image',image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()